In [14]:
import os
from glob import glob
import random
from warnings import filterwarnings
filterwarnings('ignore')


# supporting libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import random

#importing pytorch and associated libraries

import torch
import torch.nn as nn
from torchvision import transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models

In [15]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [16]:
transform = transforms.Compose([                                 
                                transforms.RandomRotation(90),
                                transforms.ToTensor()])

In [17]:
trainset = datasets.ImageFolder('C:\\Users\\ritth\\code\\Data\\datasciencebowl\\train\\train', transform = transform)
len(trainset.classes)

121

In [18]:
class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kerel_size = 3, stride = 1, padding = 0):
        super(Conv, self).__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kerel_size, stride, padding),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels),
        )
    def forward(self, x):
        return self.seq(x)

class PlankNet(nn.Module):
    def __init__(self, in_channels, num_classes, H = 128, W = 128):
        super(PlankNet, self).__init__()
        self.model = nn.Sequential(
            Conv(in_channels, 16, 4), #125
            nn.MaxPool2d(2), #62
            Conv(16, 32, 3), #60
            Conv(32, 64, 3), #58
            nn.Dropout(0.1),
            nn.MaxPool2d(2), # 29
            Conv(64, 128), # 27
            nn.Dropout(0.2),
            Conv(128, 64, 3), # 25
            Conv(64, 32, 3), # 23
            nn.Flatten(),
            nn.Linear(32*23*23 , 4096),
            nn.Linear(4096, num_classes),
        )
    def forward(self, x):
        return self.model(x)
num_classes = len(trainset.classes)
model = PlankNet(3, num_classes, 128, 128)
rand_data = torch.rand(1, 3, 128, 128)
print(model(rand_data).shape)

torch.Size([1, 121])


In [19]:
for name, param in model.named_parameters():
    print(f"{name} : {param.shape}")

model.0.seq.0.weight : torch.Size([16, 3, 4, 4])
model.0.seq.0.bias : torch.Size([16])
model.0.seq.2.weight : torch.Size([16])
model.0.seq.2.bias : torch.Size([16])
model.2.seq.0.weight : torch.Size([32, 16, 3, 3])
model.2.seq.0.bias : torch.Size([32])
model.2.seq.2.weight : torch.Size([32])
model.2.seq.2.bias : torch.Size([32])
model.3.seq.0.weight : torch.Size([64, 32, 3, 3])
model.3.seq.0.bias : torch.Size([64])
model.3.seq.2.weight : torch.Size([64])
model.3.seq.2.bias : torch.Size([64])
model.6.seq.0.weight : torch.Size([128, 64, 3, 3])
model.6.seq.0.bias : torch.Size([128])
model.6.seq.2.weight : torch.Size([128])
model.6.seq.2.bias : torch.Size([128])
model.8.seq.0.weight : torch.Size([64, 128, 3, 3])
model.8.seq.0.bias : torch.Size([64])
model.8.seq.2.weight : torch.Size([64])
model.8.seq.2.bias : torch.Size([64])
model.9.seq.0.weight : torch.Size([32, 64, 3, 3])
model.9.seq.0.bias : torch.Size([32])
model.9.seq.2.weight : torch.Size([32])
model.9.seq.2.bias : torch.Size([32])


In [20]:
def generate_csv(root,train = True, img_ext = 'jpg'):
    df = pd.DataFrame(columns = ['path', 'class'])
    if train:
        for index,label in enumerate(os.listdir(root)):
            links = glob(f"{root}/{label}/*{img_ext}")
            temp_df = pd.DataFrame({'path': links, 'class': np.ones(len(links), dtype='float32')*index})
            df = pd.concat([df, temp_df], axis = 0)
    else:
        links = glob(f"{root}/*{img_ext}")
        temp_df = pd.DataFrame({'path': links, 'class': np.ones(len(links), dtype = 'float32')})
        df = pd.concat([df, temp_df], axis = 0)
        
    return df

In [21]:
train_csv = generate_csv('C:\\Users\\ritth\\code\\Data\\datasciencebowl\\train\\train')
train_csv.head()

,path,class
0,C:\Users\ritth\code\Data\datasciencebowl\train...,0.0
1,C:\Users\ritth\code\Data\datasciencebowl\train...,0.0
2,C:\Users\ritth\code\Data\datasciencebowl\train...,0.0
3,C:\Users\ritth\code\Data\datasciencebowl\train...,0.0
4,C:\Users\ritth\code\Data\datasciencebowl\train...,0.0


In [22]:
def load_image(path , H, W):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (H,W))
    return img




class PlanktonDataset(Dataset):
    def __init__(self, df, H = 128, W = 128, transform = None):
        super(PlanktonDataset, self).__init__()
        self.df = df
        self.H = H
        self.W = W
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        path = self.df.iloc[index, 0]
        img = load_image(path, H = self.H, W = self.W)
        label = self.df.iloc[index, 1]
        img = Image.fromarray(img)
        if self.transform != None:
            img = self.transform()(img)
        else:
            img = transforms.ToTensor()(img)
        return (img, (label, path))
    
# Creating the train and test datasets.   
train_ds = PlanktonDataset(train_csv, 128, 128, transform = transform)

In [23]:
spl_idx = int(train_ds.__len__() * 0.75)
print(f"Splitting index : {spl_idx}")
train_ds, val_ds = torch.utils.data.random_split(train_ds,[spl_idx, train_ds.__len__() - spl_idx] )

Splitting index : 22752


In [24]:
BATCH_SIZE = 64
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True)
val_dl = DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle = True)

In [25]:
EPOCHS = 30
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(params = model.parameters(), lr = 1e-4)
train_loss = []
val_loss = []

In [26]:
best_loss = np.inf
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1} : \n")
    TR_LOSS = 0.0
    VAL_LOSS = 0.0
    model.train()
    
    # Train Data Forward & Backward Pass
    
    for img, (train_patch, (labels, _)) in enumerate(train_dl):
        optim.zero_grad()
        
        labels = labels.long()
        op = model(train_patch)
        tloss = criterion(op, labels)
        TR_LOSS += tloss.item()
        train_loss.append(tloss.item())
        tloss.backward()
        optim.step()
        
        if img % 100 == 99:
            print(f"         Step {img + 1} Loss : {'%.4f'%(tloss.item())}") 
    model.eval()
    
    # Validation Checking ( Only Forward Pass )
    
    with torch.no_grad():
        for img, (val_patch, (labels, _)) in enumerate(val_dl):
            
            labels = labels.long()
            op = model(val_patch)
            vloss = criterion(op, labels)
            VAL_LOSS += vloss.item()
            val_loss.append(vloss.item())
    print(f"\n     Training Loss : {'%.4f'%(TR_LOSS)}  ||  Validation Loss : {'%.4f'%(VAL_LOSS)}\n")
    
    if VAL_LOSS < best_loss :      # Model Updationg
        print("Model Updation : Success!\n", 'green')
        torch.save(model, 'best_model.pth')
        best_loss = VAL_LOSS
    else:
        print("Model Updation : Failed!\n", 'red')
print('Training completed...', 'green')  

Epoch 1 : 



TypeError: Compose.__call__() missing 1 required positional argument: 'img'